In [1]:
# Dependencies
import pandas as pd
import scipy.stats as st
import numpy as np

In [2]:
# Store filepath in a variable
us_data_path= "archive/USvideos.csv"

In [3]:
# Read the datafile with the pandas library
us_data_df = pd.read_csv(us_data_path)

In [4]:
# Looking at the data
us_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40949 entries, 0 to 40948
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   video_id                40949 non-null  object
 1   trending_date           40949 non-null  object
 2   title                   40949 non-null  object
 3   channel_title           40949 non-null  object
 4   category_id             40949 non-null  int64 
 5   publish_time            40949 non-null  object
 6   tags                    40949 non-null  object
 7   views                   40949 non-null  int64 
 8   likes                   40949 non-null  int64 
 9   dislikes                40949 non-null  int64 
 10  comment_count           40949 non-null  int64 
 11  thumbnail_link          40949 non-null  object
 12  comments_disabled       40949 non-null  bool  
 13  ratings_disabled        40949 non-null  bool  
 14  video_error_or_removed  40949 non-null  bool  
 15  de

In [5]:
# Looking at the data
us_data_df.describe()

,category_id,views,likes,dislikes,comment_count
count,40949.000000,4.094900e+04,4.094900e+04,4.094900e+04,4.094900e+04
mean,19.972429,2.360785e+06,7.426670e+04,3.711401e+03,8.446804e+03
std,7.568327,7.394114e+06,2.288853e+05,2.902971e+04,3.743049e+04
min,1.000000,5.490000e+02,0.000000e+00,0.000000e+00,0.000000e+00
25%,17.000000,2.423290e+05,5.424000e+03,2.020000e+02,6.140000e+02
50%,24.000000,6.818610e+05,1.809100e+04,6.310000e+02,1.856000e+03
75%,25.000000,1.823157e+06,5.541700e+04,1.938000e+03,5.755000e+03
max,43.000000,2.252119e+08,5.613827e+06,1.674420e+06,1.361580e+06


In [6]:
## Changing date formats and pulling out key aspects required for analysis ##

# Chaning the fromat of 'trending_date' from object to datetime
us_data_df['trending_date_formatted'] = pd.to_datetime(us_data_df['trending_date'], format='%y.%d.%m')

# Changing 'publish_time' to datetime format
us_data_df['publish_time_formatted'] = pd.to_datetime(us_data_df["publish_time"])

# Getting the day of the week the video was published and assigning it to a column
us_data_df["publish_day"] = us_data_df['publish_time_formatted'].dt.day_name()

# Getting the month the video was published and assigning it to a column
us_data_df["publish_month"] = us_data_df['publish_time_formatted'].dt.month_name()

# Getting the year the video was published and assigning it to a column
us_data_df["publish_year"] = us_data_df['publish_time_formatted'].dt.year

# Checking that the above columns have been added to the dataframe
us_data_df.head()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,...,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,trending_date_formatted,publish_time_formatted,publish_day,publish_month,publish_year
0,2kyS6SvSYSE,17.14.11,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,2017-11-13T17:13:01.000Z,SHANtell martin,748374,57527,2966,...,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...,2017-11-14,2017-11-13 17:13:01+00:00,Monday,November,2017
1,1ZAPwfrtAFY,17.14.11,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,2017-11-13T07:30:00.000Z,"last week tonight trump presidency|""last week ...",2418783,97185,6146,...,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg,False,False,False,"One year after the presidential election, John...",2017-11-14,2017-11-13 07:30:00+00:00,Monday,November,2017
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146033,5339,...,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...,2017-11-14,2017-11-12 19:05:24+00:00,Sunday,November,2017
3,puqaWrEC7tY,17.14.11,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,2017-11-13T11:00:04.000Z,"rhett and link|""gmm""|""good mythical morning""|""...",343168,10172,666,...,https://i.ytimg.com/vi/puqaWrEC7tY/default.jpg,False,False,False,Today we find out if Link is a Nickelback amat...,2017-11-14,2017-11-13 11:00:04+00:00,Monday,November,2017
4,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12T18:01:41.000Z,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095731,132235,1989,...,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...,2017-11-14,2017-11-12 18:01:41+00:00,Sunday,November,2017


In [7]:
# Ensuring the formatted dates dtype have changed
us_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40949 entries, 0 to 40948
Data columns (total 21 columns):
 #   Column                   Non-Null Count  Dtype              
---  ------                   --------------  -----              
 0   video_id                 40949 non-null  object             
 1   trending_date            40949 non-null  object             
 2   title                    40949 non-null  object             
 3   channel_title            40949 non-null  object             
 4   category_id              40949 non-null  int64              
 5   publish_time             40949 non-null  object             
 6   tags                     40949 non-null  object             
 7   views                    40949 non-null  int64              
 8   likes                    40949 non-null  int64              
 9   dislikes                 40949 non-null  int64              
 10  comment_count            40949 non-null  int64              
 11  thumbnail_link           409

In [10]:
# Counting the number of tags for each video and adding it to a new column
tag_count = []
tag = us_data_df['tags']
for word in tag:
    tag_split = word.split('|') 
    number_tags = len(tag_split)
    tag_count.append(number_tags)
tag_count

us_data_df['tag_count'] = tag_count
us_data_df.head()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,...,ratings_disabled,video_error_or_removed,description,trending_date_formatted,publish_time_formatted,publish_day,publish_month,publish_year,tag_count_append,tag_count
0,2kyS6SvSYSE,17.14.11,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,2017-11-13T17:13:01.000Z,SHANtell martin,748374,57527,2966,...,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...,2017-11-14,2017-11-13 17:13:01+00:00,Monday,November,2017,1,1
1,1ZAPwfrtAFY,17.14.11,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,2017-11-13T07:30:00.000Z,"last week tonight trump presidency|""last week ...",2418783,97185,6146,...,False,False,"One year after the presidential election, John...",2017-11-14,2017-11-13 07:30:00+00:00,Monday,November,2017,4,4
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146033,5339,...,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...,2017-11-14,2017-11-12 19:05:24+00:00,Sunday,November,2017,23,23
3,puqaWrEC7tY,17.14.11,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,2017-11-13T11:00:04.000Z,"rhett and link|""gmm""|""good mythical morning""|""...",343168,10172,666,...,False,False,Today we find out if Link is a Nickelback amat...,2017-11-14,2017-11-13 11:00:04+00:00,Monday,November,2017,27,27
4,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12T18:01:41.000Z,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095731,132235,1989,...,False,False,I know it's been a while since we did this sho...,2017-11-14,2017-11-12 18:01:41+00:00,Sunday,November,2017,14,14


In [11]:
us_data_df['tag_count'].value_counts()

1     1912
10    1447
4     1405
7     1388
13    1299
      ... 
63      16
61       9
59       5
65       2
60       1
Name: tag_count, Length: 65, dtype: int64

In [12]:
# Checking whether any video has no tag attached to it
no_tag = us_data_df.loc[us_data_df['tag_count'] == 0, :]
no_tag

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,...,ratings_disabled,video_error_or_removed,description,trending_date_formatted,publish_time_formatted,publish_day,publish_month,publish_year,tag_count_append,tag_count


In [12]:
# Filtering out those with a category_id = 43 as that category is not assignable
cleaned_us_data_df = us_data_df.loc[us_data_df['category_id'] != 43, :]

# Checking category_id = 43 has been filtered out
cleaned_us_data_df["category_id"].value_counts()

24    9964
10    6472
26    4146
23    3457
22    3210
25    2487
28    2401
1     2345
17    2174
27    1656
15     920
20     817
19     402
2      384
29      57
Name: category_id, dtype: int64

In [13]:
# Saving the cleaned file to a csv
cleaned_us_data_df.to_csv("output/cleaned_us_data.csv", index=False)